In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
o1 = pd.read_csv('experimentation/observations_round_4_day_1.csv')
o2 = pd.read_csv('experimentation/observations_round_4_day_2.csv')
o3 = pd.read_csv('experimentation/observations_round_4_day_3.csv')

p1 = pd.read_csv('experimentation/prices_round_4_day_1.csv', sep=';')
p1 = [p1[p1['product'] == 'MAGNIFICENT_MACARONS']]
p2 = pd.read_csv('experimentation/prices_round_4_day_2.csv', sep=';')
p2 = [p2[p2['product'] == 'MAGNIFICENT_MACARONS']]
p3 = pd.read_csv('experimentation/prices_round_4_day_3.csv', sep=';')
p3 = [p3[p3['product'] == 'MAGNIFICENT_MACARONS']]


p1 = pd.concat(p1).set_index('timestamp')
p2 = pd.concat(p2).set_index('timestamp')
p3 = pd.concat(p3).set_index('timestamp')
o1 = o1.set_index('timestamp')
o2 = o2.set_index('timestamp')
o3 = o3.set_index('timestamp')


In [3]:
maxBid = lambda df: df.assign(max_bid=df[['bid_price_1', 'bid_price_2', 'bid_price_3']].max(axis=1, skipna=True))
minAsk = lambda df: df.assign(min_ask=df[['ask_price_1', 'ask_price_2', 'ask_price_3']].min(axis=1, skipna=True))

In [4]:
p1 = maxBid(p1)
p1 = minAsk(p1)
p2 = maxBid(p2)
p2 = minAsk(p2)
p3 = maxBid(p3)
p3 = minAsk(p3)


In [7]:
import pandas as pd

def simulate_sugar_strategy(df_sugar, price_col='sugar_price', max_pos=75):
    """
    Simulate a sugar momentum strategy:
    - At each timestamp t, compare price at t vs t-3 (3 rows ago).
    - If abs(delta) > 3: adjust position by +1 (if price rose) or -1 (if price fell), capped at ±max_pos.
    - Otherwise, clear full position to 0.
    - Record each trade and compute cumulative PnL.
    
    Returns:
      trades_df: DataFrame with columns ['timestamp','quantity','price','position']
      total_pnl: float, sum of all trade PnL
    """
    o1['prev_price'] = o1[price_col].shift(3)
    
    position = 0
    pnl = 0.0
    trades = []
    
    for _, row in o1.iterrows():
        t = row['timestamp']
        price = row[price_col]
        prev_price = row['prev_price']
        
        # Determine action
        if pd.notnull(prev_price) and abs(price - prev_price) > 3:
            direction = 1 if (price - prev_price) > 0 else -1
            new_pos = position + direction
            new_pos = min(max(new_pos, -max_pos), max_pos)  # clip to ±max_pos
            qty = new_pos - position
        else:
            qty = -position
            new_pos = 0
        
        if qty != 0:
            trades.append({
                'timestamp': t,
                'quantity': qty,
                'price': price,
                'position': new_pos
            })
            pnl -= qty * price
            position = new_pos
    
    trades_df = pd.DataFrame(trades)
    return trades_df, pnl


In [8]:
trades_df, total_pnl = simulate_sugar_strategy(o1['sugarPrice'], price_col='sugar_price', max_pos=75)
print(f"Total PnL: {total_pnl:.2f}")
trades_df.head()

NameError: name 'df' is not defined